# 1. LLM - RAG (langchain)
Get API keys in enironment variables

In [1]:
import dotenv
dotenv.load_dotenv()

True

Get gemini-pro as LLM

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro")

result = llm.invoke("who are you?")
result.content


'I am Gemini, a large multi-modal model, trained by Google.'

Load the webpage

In [8]:
from langchain.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))
    )
)
docs = loader.load()
print(docs[0].page_content[:100])




      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |


Split, index the contents & store in a vectorDB

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits  = text_splitter.split_documents(docs)
embedding = HuggingFaceEmbeddings()
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)

/Users/saeed.anwar/Projects/venvs/rag_llm_chatbot/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Prepare a retirever to retrive from the vectorstore

In [18]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")
retrieved_docs[0].page_content

'Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'

Now generate Answers using the retriever & LLM

In [25]:
# prompt = """
# You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
# Question: {question} 
# Context: {context} 
# Answer:
# """

# load the prompt from the langchain hub
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt.invoke({"question": "filler question?", "context": "filler_context"}).to_messages()

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question? \nContext: filler_context \nAnswer:")]

In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [31]:
for chunk in rag_chain.stream("What is a generative agent?"):
    print(chunk, end="", flush=True)

Generative agents are virtual characters controlled by LLM agents that interact in a sandbox environment. They create believable simulations of human behavior, combining LLM with memory, planning, and reflection mechanisms. Generative agents enable agents to behave based on past experiences and interact with other agents.

# Let's try with a different document (Agricultural handbook)

In [41]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Hand Book of Agricultural Technology.pdf")
pages = loader.load_and_split()

pages = pages[17:26] # select a range of pages (rice cultivation)

In [42]:
from langchain_community.vectorstores import chroma
from langchain.embeddings import HuggingFaceEmbeddings

vectorstore = Chroma.from_documents(documents=pages, embedding=HuggingFaceEmbeddings())
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

retrieved_docs = retriever.invoke("What is the grain yield of BRRI hybrid dhan2")
retrieved_docs[0].page_content

/Users/saeed.anwar/Projects/venvs/rag_llm_chatbot/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


'09Grain type:  Medium slender. Plant height:  100 cm. \nGrain yield:  7.0 t/ha (Boro) and 5.0 t/ha (Aus). \nDuration:  145 days (Boro) and 105 days (Aus).  \nGrain type:  Medium slender. Plant height:  115 cm. \nResistant to blast. It is grown in Aman season. Grain \nyield:  4.0 t/ha. Duration:  106 days\nGrain type:  Medium slender. Plant height:  100 cm. \nIt is grown in Aman season. Grain yield:  4.0 t/ha. \nDuration:  105 days.\nGrain type:  Medium slender. Plant height:  115 cm. \nIt is grown in Boro season. Grain yield:  7.1 t/ha. \nDuration:  153 days.\nGrain type:  Long fine grain. It is early maturing \nhigh yielding variety (HYV). Resistant to blast. It is grown in Aman season. Grain yield:  5.2 t/ha. \nDuration:  110-115 days.  \nGrain type:  Medium fine grain. High yielding \nvariety (HYV). Salt tolerant. Resistant to Bacterial leaf blight. It is grown in Boro season. Grain yield:  \n5.5 t/ha. Duration:  130-135 days.\nBRRI \ndhan55\n(2011)\nBRRI \ndhan56 \n(2011)\nBRRI \n

In [43]:
retrieved_docs = retriever.invoke("What is the grain yield of BRRI hybrid dhan2")
retrieved_docs

[Document(page_content='09Grain type:  Medium slender. Plant height:  100 cm. \nGrain yield:  7.0 t/ha (Boro) and 5.0 t/ha (Aus). \nDuration:  145 days (Boro) and 105 days (Aus).  \nGrain type:  Medium slender. Plant height:  115 cm. \nResistant to blast. It is grown in Aman season. Grain \nyield:  4.0 t/ha. Duration:  106 days\nGrain type:  Medium slender. Plant height:  100 cm. \nIt is grown in Aman season. Grain yield:  4.0 t/ha. \nDuration:  105 days.\nGrain type:  Medium slender. Plant height:  115 cm. \nIt is grown in Boro season. Grain yield:  7.1 t/ha. \nDuration:  153 days.\nGrain type:  Long fine grain. It is early maturing \nhigh yielding variety (HYV). Resistant to blast. It is grown in Aman season. Grain yield:  5.2 t/ha. \nDuration:  110-115 days.  \nGrain type:  Medium fine grain. High yielding \nvariety (HYV). Salt tolerant. Resistant to Bacterial leaf blight. It is grown in Boro season. Grain yield:  \n5.5 t/ha. Duration:  130-135 days.\nBRRI \ndhan55\n(2011)\nBRRI \nd

In [52]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
for chunk in rag_chain.stream("What is the rice with most yield?"):
    print(chunk, end="", flush=True)

The rice with the highest yield is BRRI dhan29, which has a grain yield of 7.5 t/ha. It is a medium slender and white rice variety that is mainly grown in the Boro season.

In [56]:
for chunk in rag_chain.stream("What are the varities that have slender grain types?"):
    print(chunk, end="", flush=True)

BRRI dhan36 and Bina dhan7 are two varieties that have long slender grain types.

Accuracy is not that good. since this is similar to tabular data and they all are related in the same context. so page chunks does not help either